In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hemul
from hemul import loader
he = loader.load()
from hemul import heaan
from muxcnn.resnet_HEAAN import ResNetHEAAN
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv
from muxcnn.utils import get_channel_last, get_conv_params
from muxcnn.hecnn_par import MultParPack
from muxcnn.hecnn_par import forward_convbn_par

Using CPU version HEAAN
Using CPU version HEAAN
Using CPU version HEAAN
Using CPU version HEAAN


## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

img_tensor = load_img("./cute.jpg", hi=32, wi=32)

## Image

In [4]:
imgl = get_channel_last(img_tensor[0].detach().numpy())
ki = 1 # initial ki
hi, wi, ch = imgl.shape

# FHE

In [ ]:
rot_l = [2**i for i in range(15)]

# prepare for all rotations
rot_l = rot_l + [2**15-1, 
         2**15-33, 2**15-32, 2**15-31,
         2**15-17, 2**15-16, 2**15-15, 
         2**15-9,2**15-8, 2**15-7] + [3,5,7,9,15,17, 31, 33]

# try 40 & 800
hec = heaan.HEAANContext(15, 40, 800, boot=False, load_sk=True, rot_l=rot_l)

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt


In [ ]:
fhemodel = ResNetHEAAN(model, hec)

ctxt = fhemodel.pack_img_ctxt(img_tensor)

# forward early

In [ ]:
ctxt0, outs0 = fhemodel.forward_early(ctxt, 1, 32, 32)

In [ ]:
dec = hec.decrypt(ctxt0)
print(dec[::1000])

In [ ]:
hec.rescale(ctxt0)

In [ ]:
ctxt1, outs1 = fhemodel.forward_bb(model.layer1[0], ctxt0, outs0)

In [11]:
ctxt2, outs2 = fhemodel.forward_bb(model.layer2[0], ctxt1, outs1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
ct_a -160 360 [-7.55495086e+157 -5.19874646e+156 -1.43452015e+158  5.92495583e+157
  3.14469788e+158  7.73574841e+156  1.88512267e+158  3.21361038e+157
  5.50819414e+157 -3.15773732e+158 -2.71470983e+158 -1.31103105e+158
  7.90035679e+157  6.77592755e+157  6.99490466e+157  1.68226352e+158
  2.11461367e+158 -1.94913619e+157  2.39647162e+158 -1.19243521e+158
  1.16999473e+158  8.14904659e+157 -1.97164266e+157  1.67444649e+158
  1.86504860e+158  1.89382521e+158 -6.20627674e+157  2.36085322e+158
 -4.67296013e+156  1.73012432e+158  2.73635047e+157 -1.07806796e+158
 -3.90507233e+157]
ct_d 40 280
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
40 320
APPR 40 280
[0.97729385 0.97729385 0.97729385 ... 0.97729385 0.97729385 0.97729385]
min max 0.9772938467112908 0.9772938467112908
APPR 40 80
[0.48910557 0.489


KeyboardInterrupt



In [ ]:
ctxt3, outs3 = fhemodel.forward_bb(model.layer3[0], ctxt2, outs2)

In [5]:
# early conv and bn
_, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins0)

In [6]:
# Muxed
U, ins1, outs1 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins1)

out1, un1 = forward_convbn_par(model.conv1, 
                              model.bn1, ct_a, ins1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
2.7468 s


## Activation